# TSA Chapter 9: Changepoint Detection

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch9_changepoint_detection/TSA_ch9_changepoint_detection.ipynb)

This notebook generates the automatic changepoint detection visualization showing piecewise linear trend estimation with identified changepoints.

In [ ]:
!pip install numpy pandas matplotlib -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Color scheme
BLUE    = '#1A3A6E'
RED     = '#DC3545'
GREEN   = '#2E7D32'
ORANGE  = '#E67E22'
GRAY    = '#666666'
PURPLE  = '#8E44AD'

# Transparent backgrounds
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['savefig.facecolor'] = 'none'
plt.rcParams['savefig.transparent'] = True

# No top/right spines
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# General styling
plt.rcParams['axes.grid'] = False
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Helvetica', 'Arial', 'DejaVu Sans']
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9
plt.rcParams['legend.facecolor'] = 'none'
plt.rcParams['legend.framealpha'] = 0
plt.rcParams['axes.linewidth'] = 0.6
plt.rcParams['lines.linewidth'] = 1.0


def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    try:
        charts_path = os.path.join('..', '..', 'charts', name)
        fig.savefig(f'{charts_path}.pdf', bbox_inches='tight', transparent=True, dpi=150)
        fig.savefig(f'{charts_path}.png', bbox_inches='tight', transparent=True, dpi=150)
    except Exception:
        pass
    print(f'Saved: {name}.pdf + .png')


def legend_outside(ax, ncol=3):
    """Place legend outside bottom of plot."""
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), ncol=ncol, frameon=False)

In [ ]:
# Chart: ch9_changepoint_detection
# Automatic changepoint detection in trend
np.random.seed(42)
n_cp = 500
t_cp = np.arange(n_cp)

# Build piecewise linear trend
changepoints = [120, 250, 380]
slopes = [0.10, -0.05, 0.15, 0.02]
trend_cp = np.zeros(n_cp)
current_level = 10.0
current_slope = slopes[0]
cp_idx = 0
for i in range(n_cp):
    if cp_idx < len(changepoints) and i == changepoints[cp_idx]:
        cp_idx += 1
        current_slope = slopes[cp_idx]
    trend_cp[i] = current_level
    current_level += current_slope

y_cp = trend_cp + np.random.normal(0, 1.5, n_cp)

fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(t_cp, y_cp, color=GRAY, linewidth=0.8, alpha=0.6, label='Observed')
ax.plot(t_cp, trend_cp, color=BLUE, linewidth=2.5, label='Estimated Trend')

for cp in changepoints:
    ax.axvline(x=cp, color=RED, linestyle='--', linewidth=2, alpha=0.7)
    ax.scatter([cp], [trend_cp[cp]], s=120, color=RED, zorder=5, edgecolors='white')
# Dummy for legend
ax.scatter([], [], s=100, color=RED, label='Changepoints')

# Annotate slopes
segments = [(0, changepoints[0]), (changepoints[0], changepoints[1]),
            (changepoints[1], changepoints[2]), (changepoints[2], n_cp - 1)]
for (s_start, s_end), slope in zip(segments, slopes):
    mid = (s_start + s_end) // 2
    ax.annotate(f'slope = {slope:.2f}', xy=(mid, trend_cp[mid] + 3),
                fontsize=9, color=BLUE, ha='center',
                bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.7, edgecolor=BLUE))

ax.set_title('Automatic Changepoint Detection in Trend\\n'
              r'$g(t) = (k + \mathbf{a}(t)^T \delta) \cdot t + (m + \mathbf{a}(t)^T \gamma)$',
              fontweight='bold')
ax.set_xlabel('Time')
ax.set_ylabel('Value')
legend_outside(ax, ncol=3)

plt.tight_layout()
save_chart(fig, 'ch9_changepoint_detection')
plt.show()